# Data Preparation

In [22]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

train_dir = './train_images'
test_dir = './test_images'

# Define the transformations for augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(), 
        transforms.RandomResizedCrop(36),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0,),std=(1,))
    ]),
    'test':transforms.Compose([
        transforms.Grayscale(), 
        transforms.ToTensor(), 
        transforms.Normalize(mean=(0,),std=(1,))])        
}
train_data_augmented = torchvision.datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_data = torchvision.datasets.ImageFolder(test_dir, transform=data_transforms['test'])

train_data_size = len(train_data_augmented)
test_data_size = len(test_data)

There are 26950 images without faces and 64770 with faces in the training dataset

In [5]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from torchsampler import ImbalancedDatasetSampler

valid_size = 0.2
batch_size = 32
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(train_data_augmented, batch_size=batch_size, sampler=ImbalancedDatasetSampler(train_data_augmented), num_workers=2)
valid_loader = torch.utils.data.DataLoader(train_data_augmented, batch_size=batch_size, sampler=ImbalancedDatasetSampler(train_data_augmented), num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=2)
classes = ('noface','face')

# Define the model

In [8]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16) # Add a new fully connected layer
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = Net().to(device)

# Training

In [14]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4) # Use Adam optimization

scheduler = StepLR(optimizer, step_size=5, gamma=0.8)

n_epochs = 32
epsilon = 0.1
early_stop = False

for epoch in range(1, n_epochs+1):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'Epoch [{epoch}/{n_epochs}], Batch [{i + 1}/{len(train_loader)}], Loss: {running_loss / 2000:.3f}, Learning Rate: {scheduler.get_last_lr()[0]:.6f}')
            # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            if (running_loss / 2000) < epsilon:
                early_stop = True
                break
            running_loss = 0.0
    
    if early_stop:
        break
    
    # Step the learning rate scheduler at the end of each epoch
    scheduler.step()
# need to implement stop if the accuracy doesn't go down enough

print('Finished Training')

Epoch [1/64], Batch [2000/2867], Loss: 0.381, Learning Rate: 0.001000
Epoch [2/64], Batch [2000/2867], Loss: 0.223, Learning Rate: 0.001000
Epoch [3/64], Batch [2000/2867], Loss: 0.181, Learning Rate: 0.001000
Epoch [4/64], Batch [2000/2867], Loss: 0.162, Learning Rate: 0.001000
Epoch [5/64], Batch [2000/2867], Loss: 0.145, Learning Rate: 0.001000
Epoch [6/64], Batch [2000/2867], Loss: 0.120, Learning Rate: 0.000500
Epoch [7/64], Batch [2000/2867], Loss: 0.111, Learning Rate: 0.000500
Epoch [8/64], Batch [2000/2867], Loss: 0.108, Learning Rate: 0.000500
Epoch [9/64], Batch [2000/2867], Loss: 0.106, Learning Rate: 0.000500
Epoch [10/64], Batch [2000/2867], Loss: 0.102, Learning Rate: 0.000500
Epoch [11/64], Batch [2000/2867], Loss: 0.090, Learning Rate: 0.000250
Epoch [12/64], Batch [2000/2867], Loss: 0.089, Learning Rate: 0.000250
Epoch [13/64], Batch [2000/2867], Loss: 0.089, Learning Rate: 0.000250
Epoch [14/64], Batch [2000/2867], Loss: 0.087, Learning Rate: 0.000250
Epoch [15/64], 

In [15]:
PATH = './lr_scheduling_net.pth'
torch.save(net.state_dict(), PATH)

In [14]:
# load the trained model to avoid retrain from scratch
PATH = './lr_scheduling_net.pth'
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

# Test the model

In [15]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 7628 test images: {100 * correct / total:.2f}')

Accuracy of the network on the 7648 test images: 98.09
